In [1]:
import pymongo, datetime, dotenv, os, json, copy
import pandas as pd
import numpy as np

# ENVIROMENT VARIABLES
dotenv.load_dotenv() 
DB_NAME = os.environ["DB_NAME"]
INFLUENCERS_COLLECTION = os.environ["INFLUENCERS_COLLECTION"]
LOG_COLLETION = os.environ["LOG_COLLETION"]
NEWS_COLLECTION = os.environ["NEWS_COLLECTION"]
INSTAGRAM_COLLECTION = os.environ["INSTAGRAM_COLLECTION"]
TREND_COLLECTION = os.environ["TREND_COLLECTION"]
MONGO_CONNECT_STRING = os.environ["MONGO_CONNECT_STRING"]

In [2]:
# MongoDB Connector
mongo_client = pymongo.MongoClient(MONGO_CONNECT_STRING, serverSelectionTimeoutMS=1)

# Set DB 
db = mongo_client[DB_NAME]

# List of influencers
influencers_list = list(db[INFLUENCERS_COLLECTION].find())
display(influencers_list[:2])
print("...")
display(influencers_list[-2:])

[{'_id': ObjectId('63c401049ebdf2a03c4c17d4'),
  'username': 'khaby00',
  'names': ['khaby00', 'Khaby Lame'],
  'position': {'$numberInt': '1'},
  'acquisition_datetime': datetime.datetime(2023, 1, 15, 13, 34, 56, 386000)},
 {'_id': ObjectId('63c401049ebdf2a03c4c17d5'),
  'username': 'chiaraferragni',
  'names': ['chiaraferragni', 'Chiara Ferragni', 'ferragnez'],
  'position': {'$numberInt': '2'},
  'acquisition_datetime': datetime.datetime(2023, 1, 15, 13, 34, 56, 386000)}]

...


[{'_id': ObjectId('63c401049ebdf2a03c4c1836'),
  'username': 'fishball_sg',
  'names': ['fishball_sg', 'Fishball'],
  'position': {'$numberInt': '99'},
  'acquisition_datetime': datetime.datetime(2023, 1, 15, 13, 34, 56, 386000)},
 {'_id': ObjectId('63c401049ebdf2a03c4c1837'),
  'username': 'lorinsigneofficial',
  'names': ['lorinsigneofficial', 'Lorenzo Insigne'],
  'position': {'$numberInt': '100'},
  'acquisition_datetime': datetime.datetime(2023, 1, 15, 13, 34, 56, 386000)}]

In [3]:
# List of log to validate quality of data: contains datetime related to all VALID acquisitions
valid_acquisition_datetime = list(db[LOG_COLLETION].find())
valid_acquisition_datetime = [x["datetime"] for x in valid_acquisition_datetime]
display(valid_acquisition_datetime)


[datetime.datetime(2023, 1, 15, 13, 34, 56, 386000),
 datetime.datetime(2023, 1, 15, 13, 34, 56, 386000),
 datetime.datetime(2023, 1, 15, 13, 34, 56, 386000),
 datetime.datetime(2023, 1, 15, 13, 34, 56, 386000)]

In [4]:
# Define function to return all document related to a given user in a given collection
def getUserData(collection_name, username):
        collection = db[collection_name]
        return list(collection.find({"username":username}))

In [5]:
# Example of integration for the first influencer for the first day of 2022

username = influencers_list[0]["username"]

# Get all data from all collections
ansa = getUserData(NEWS_COLLECTION, username) 
instagram = getUserData(INSTAGRAM_COLLECTION, username) 
trends = getUserData(TREND_COLLECTION, username)
print("Username: {}:\n- {} ansa entry\n- {} instagram entry\n- {} trends entry".format(username, len(ansa), len(instagram), len(trends)))

# Remove invalid data (acquisition datetime not in log collection)
ansa = [x for x in ansa if x["acquisition_datetime"] in valid_acquisition_datetime] 
instagram = [x for x in instagram if x["acquisition_datetime"] in valid_acquisition_datetime] 
trends = [x for x in trends if x["acquisition_datetime"] in valid_acquisition_datetime]
print("Username: {}:\n- {} ansa entry\n- {} instagram entry\n- {} trends entry".format(username, len(ansa), len(instagram), len(trends)))

daily_old_entry = {}
trends_element = trends[0]

new_entry = {}
new_entry["username"] = username
new_entry["day"] = trends_element["week"] + datetime.timedelta(days=0)

# Instagram
instagram_data = [raw_data for raw_data in instagram if new_entry["day"].date() == raw_data["timestamp"].date()]
if len(instagram_data) == 0:
    new_entry["followers"] = None
    new_entry["following"] = None
    new_entry["post"] = None
    new_entry["followers_delta"] = None
    new_entry["following_delta"] = None
    new_entry["post_delta"] = None
else:
    new_entry["followers"] = instagram_data[0]["followers"]
    new_entry["following"] = instagram_data[0]["following"]
    new_entry["post"] = instagram_data[0]["post_count"]
    if daily_old_entry != {} and daily_old_entry["followers"] is not None:
        new_entry["followers_delta"] = new_entry["followers"] - daily_old_entry["followers"]
        new_entry["following_delta"] = new_entry["following"] - daily_old_entry["following"]
        new_entry["post_delta"] = new_entry["post"] - daily_old_entry["post"]
    else:
        new_entry["followers_delta"] = None
        new_entry["following_delta"] = None
        new_entry["post_delta"] = None

# Google trends
new_entry["trends"] = trends_element["max_trend"]

# Ansa news
ansa_articles = [{"title":raw_data["title"], "inTitle":raw_data["inTitle"]} for raw_data in ansa if new_entry["day"].date() >= raw_data["timestamp"].date()]
new_entry["ansa"] = len(ansa_articles)
if daily_old_entry != {}:
    new_entry["ansa_delta"] = new_entry["ansa"] - daily_old_entry["ansa"] 
else:
    new_entry["ansa_delta"] = 0

new_entry["ansa_articles"] = [{"title":raw_data["title"], "inTitle":raw_data["inTitle"]} for raw_data in ansa if new_entry["day"].date() == raw_data["timestamp"].date()]


print("\nRAW DATA:")
display(instagram_data)
display(trends_element)
display(ansa_articles)

print("\nINTEGRATED DATA:")
display(new_entry)

Username: khaby00:
- 17 ansa entry
- 364 instagram entry
- 54 trends entry
Username: khaby00:
- 17 ansa entry
- 364 instagram entry
- 54 trends entry

RAW DATA:


[{'_id': ObjectId('63c401a09ebdf2a03c4c2465'),
  'acquisition_datetime': datetime.datetime(2023, 1, 15, 13, 34, 56, 386000),
  'username': 'khaby00',
  'followers': 64717175,
  'following': 930,
  'post_count': 215,
  'timestamp': datetime.datetime(2022, 1, 2, 0, 0)}]

{'_id': ObjectId('63c401a69ebdf2a03c4cb154'),
 'week': datetime.datetime(2022, 1, 2, 0, 0),
 'username': 'khaby00',
 'max_trend': 18,
 'acquisition_datetime': datetime.datetime(2023, 1, 15, 13, 34, 56, 386000)}

[]


INTEGRATED DATA:


{'username': 'khaby00',
 'day': datetime.datetime(2022, 1, 2, 0, 0),
 'followers': 64717175,
 'following': 930,
 'post': 215,
 'followers_delta': None,
 'following_delta': None,
 'post_delta': None,
 'trends': 18,
 'ansa': 0,
 'ansa_delta': 0,
 'ansa_articles': []}

In [6]:
# Data Integration

# Datasets
influencers_dataset = []
articles_dataset = []
dataset = []

for influencer in influencers_list:
    
    username = influencer["username"]

    # Get all data from all collections
    ansa = getUserData(NEWS_COLLECTION, username) 
    instagram = getUserData(INSTAGRAM_COLLECTION, username) 
    trends = getUserData(TREND_COLLECTION, username)
  
    # Remove invalid data (acquisition datetime not in log collection)
    ansa = [x for x in ansa if x["acquisition_datetime"] in valid_acquisition_datetime] 
    instagram = [x for x in instagram if x["acquisition_datetime"] in valid_acquisition_datetime] 
    trends = [x for x in trends if x["acquisition_datetime"] in valid_acquisition_datetime]
    
    #print("Username: {}:\n- {} ansa entry\n- {} instagram entry\n- {} trends entry".format(username, len(ansa), len(instagram), len(trends)))

    daily_old_entry = {}
    entry_count = 0
    
    for trends_element in trends:
        for i in range(7):
            if trends_element["week"] + datetime.timedelta(days=i) < datetime.datetime(year=2023,month=1, day=1, hour=0,minute=0):

                new_entry = {}
                new_entry["username"] = username
                new_entry["day"] = trends_element["week"] + datetime.timedelta(days=i)

                # Instagram
                instagram_data = [raw_data for raw_data in instagram if new_entry["day"].date() == raw_data["timestamp"].date()]
                if len(instagram_data) == 0:
                    continue
                else:
                    new_entry["followers"] = instagram_data[0]["followers"]
                    new_entry["following"] = instagram_data[0]["following"]
                    new_entry["post"] = instagram_data[0]["post_count"]
                    if daily_old_entry != {} and daily_old_entry["followers"] is not None:
                        new_entry["followers_delta"] = new_entry["followers"] - daily_old_entry["followers"]
                        new_entry["following_delta"] = new_entry["following"] - daily_old_entry["following"]
                        new_entry["post_delta"] = new_entry["post"] - daily_old_entry["post"]
                    else:
                        new_entry["followers_delta"] = None
                        new_entry["following_delta"] = None
                        new_entry["post_delta"] = None

                # Google trends
                new_entry["trends"] = trends_element["max_trend"]

                # Ansa news
                ansa_articles = [{"title":raw_data["title"], "inTitle":raw_data["inTitle"]} for raw_data in ansa if new_entry["day"].date() >= raw_data["timestamp"].date()]
                new_entry["ansa"] = len(ansa_articles)
                if daily_old_entry != {}:
                    new_entry["ansa_delta"] = new_entry["ansa"] - daily_old_entry["ansa"] 
                else:
                    new_entry["ansa_delta"] = 0

                ansa_articles = [{"username": username, "day": new_entry["day"].strftime('%m-%d-%Y'),"title":raw_data["title"], "inTitle":raw_data["inTitle"]} for raw_data in ansa if new_entry["day"].date() == raw_data["timestamp"].date()]
                for article in ansa_articles:
                    articles_dataset.append(article)
                
                daily_old_entry = new_entry
                new_entry["day"] = new_entry["day"].strftime('%m-%d-%Y')
                influencers_dataset.append(new_entry)
                new_entry = copy.copy(new_entry)
                new_entry["ansa_articles"] = ansa_articles
                dataset.append(new_entry)
                entry_count += 1
    
    print("Added {} rows for {}".format(entry_count, username))

# Migrate to Pandas Dataframe
influencers_dataset = pd.DataFrame.from_dict(influencers_dataset)
articles_dataset = pd.DataFrame.from_dict(articles_dataset)

print("\nDataset:")
display(dataset[:2])
print("...")
display(dataset[-2:])
print("\nInfluencers dataset:")
display(influencers_dataset)
print("\nArticles dateset:")
display(articles_dataset)

Added 363 rows for khaby00
Added 363 rows for chiaraferragni
Added 362 rows for gianlucavacchi
Added 362 rows for iammichelemorroneofficial
Added 363 rows for valeyellow46
Added 363 rows for fedez
Added 357 rows for mb459
Added 363 rows for belenrodriguezreal
Added 357 rows for gianluigibuffon
Added 357 rows for mrancelotti
Added 355 rows for andreapirlo21
Added 363 rows for dilettaleotta
Added 362 rows for itsmarziapie
Added 356 rows for marco_verratti92
Added 362 rows for elettralamborghini
Added 357 rows for francesconappoph
Added 362 rows for marianodivaio
Added 362 rows for maneskinofficial
Added 363 rows for alex_mucci
Added 360 rows for fedevigevani
Added 363 rows for vismaramartina
Added 363 rows for real_brown
Added 356 rows for mariotestino
Added 357 rows for bonuccileo19
Added 358 rows for therealhunzigram
Added 357 rows for alessiamarcuzzi
Added 357 rows for stewel92
Added 363 rows for giuliadelellis103
Added 358 rows for ludovicoaldasio
Added 355 rows for giorgiochiellini


[{'username': 'khaby00',
  'day': '01-02-2022',
  'followers': 64717175,
  'following': 930,
  'post': 215,
  'followers_delta': None,
  'following_delta': None,
  'post_delta': None,
  'trends': 18,
  'ansa': 0,
  'ansa_delta': 0,
  'ansa_articles': []},
 {'username': 'khaby00',
  'day': '01-03-2022',
  'followers': 64794028,
  'following': 930,
  'post': 216,
  'followers_delta': 76853,
  'following_delta': 0,
  'post_delta': 1,
  'trends': 18,
  'ansa': 0,
  'ansa_delta': 0,
  'ansa_articles': []}]

...


[{'username': 'lorinsigneofficial',
  'day': '12-23-2022',
  'followers': 2264831,
  'following': 218,
  'post': 298,
  'followers_delta': -334,
  'following_delta': 0,
  'post_delta': 0,
  'trends': 4,
  'ansa': 79,
  'ansa_delta': 0,
  'ansa_articles': []},
 {'username': 'lorinsigneofficial',
  'day': '12-31-2022',
  'followers': 2260659,
  'following': 221,
  'post': 298,
  'followers_delta': -4172,
  'following_delta': 3,
  'post_delta': 0,
  'trends': 2,
  'ansa': 79,
  'ansa_delta': 0,
  'ansa_articles': []}]


Influencers dataset:


,username,day,followers,following,post,followers_delta,following_delta,post_delta,trends,ansa,ansa_delta
0,khaby00,01-02-2022,64717175,930,215,NaN,NaN,NaN,18,0,0
1,khaby00,01-03-2022,64794028,930,216,76853.0,0.0,1.0,18,0,0
2,khaby00,01-04-2022,64955821,930,216,161793.0,0.0,0.0,18,0,0
3,khaby00,01-05-2022,65108863,930,217,153042.0,0.0,1.0,18,0,0
4,khaby00,01-06-2022,65236713,930,217,127850.0,0.0,0.0,18,0,0
...,...,...,...,...,...,...,...,...,...,...,...
35974,lorinsigneofficial,12-20-2022,2265912,218,298,-340.0,0.0,0.0,4,79,0
35975,lorinsigneofficial,12-21-2022,2265569,218,298,-343.0,0.0,0.0,4,79,0
35976,lorinsigneofficial,12-22-2022,2265165,218,298,-404.0,0.0,0.0,4,79,0
35977,lorinsigneofficial,12-23-2022,2264831,218,298,-334.0,0.0,0.0,4,79,0



Articles dateset:


,username,day,title,inTitle
0,khaby00,03-02-2022,Premio La Moda veste la Pace al tiktoker Khaby...,True
1,khaby00,04-13-2022,"Meno tweet meno post, influencer poco esposti ...",False
2,khaby00,05-27-2022,TikToker Khaby Lame inaugura biblioteca per ba...,True
3,khaby00,06-23-2022,Khaby Lame il personaggio più seguito al mondo...,True
4,khaby00,06-23-2022,Ora e' Khaby Lame il tiktoker piu' seguito al ...,True
...,...,...,...,...
2964,lorinsigneofficial,08-07-2022,"Serie A: da De Ligt a Koulibaly, esodo di camp...",False
2965,lorinsigneofficial,08-07-2022,"Primo gol di Insigne col Toronto, a segno pure...",False
2966,lorinsigneofficial,08-07-2022,"Calcio: Napoli bloccato da Espanyol, aspetta m...",False
2967,lorinsigneofficial,08-08-2022,"Insigne: 'A Toronto tutto nuovo, ma tiferò sem...",False


In [7]:
#Store dataset in CSV and JSON
articles_dataset.to_csv("dataset/articles_dataset.csv")
influencers_dataset.to_csv("dataset/influencers_dataset.csv")
with open("dataset/dataset.json", "w") as f:
    json.dump(dataset, f)
 


/home/simon/.local/lib/python3.10/site-packages/pandas/core/internals/blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [8]:
# Data quality completeness evaluation
print("Data quality completeness evaluation - InfluencersDataset\n")

# Dataset size
n_rows = influencers_dataset.shape[0]
n_columns = influencers_dataset.shape[1]
print("Number of rows: {}".format(n_rows))
print("Number of columns: {}".format(n_columns))

# Object completeness (theorically one row for each day of the year for each influencer)
print("\nObject completeness: {}%".format(n_rows*100/(365*100)))

# Tuple completeness
tupleCompleteness = {}
print("\nTuple completeness (in %)")
for index, row in influencers_dataset.iterrows():
    key = str(int(100 - (row.isna().sum()*100)/n_columns)) + "%"
    if key in tupleCompleteness:
        tupleCompleteness[key] += 1
    else:
        tupleCompleteness[key] = 1
for key in tupleCompleteness:
    print("{}% tuple with {} of completeness".format(round(tupleCompleteness[key]*100/n_rows,3), key))

# Attribute completeness
print("\nAttribute completeness (in %)")
print(round(100 - influencers_dataset.isna().sum()*100/n_rows,3))

# Table completeness
print("\nTable completeness: {}%".format(100-round(influencers_dataset.isna().sum().sum()*100/(n_columns*n_rows),3)))


Data quality completeness evaluation - InfluencersDataset

Number of rows: 35979
Number of columns: 11

Object completeness: 98.57260273972602%

Tuple completeness (in %)
0.278% tuple with 72% of completeness
99.722% tuple with 100% of completeness

Attribute completeness (in %)
username           100.000
day                100.000
followers          100.000
following          100.000
post               100.000
followers_delta     99.722
following_delta     99.722
post_delta          99.722
trends             100.000
ansa               100.000
ansa_delta         100.000
dtype: float64

Table completeness: 99.924%


In [9]:
# Data quality completeness evaluation
print("Data quality completeness evaluation - ArticlesDataset\n")

# Dataset size
n_rows = articles_dataset.shape[0]
n_columns = articles_dataset.shape[1]
print("Number of rows: {}".format(n_rows))
print("Number of columns: {}".format(n_columns))

# Tuple completeness
tupleCompleteness = {}
print("\nTuple completeness (in %)")
for index, row in articles_dataset.iterrows():
    key = str(int(100 - (row.isna().sum()*100)/n_columns)) + "%"
    if key in tupleCompleteness:
        tupleCompleteness[key] += 1
    else:
        tupleCompleteness[key] = 1
for key in tupleCompleteness:
    print("{}% tuple with {} of completeness".format(round(tupleCompleteness[key]*100/n_rows,3), key))

# Attribute completeness
print("\nAttribute completeness (in %)")
print(round(100 - articles_dataset.isna().sum()*100/n_rows,3))

# Table completeness
print("\nTable completeness: {}%".format(100-round(articles_dataset.isna().sum().sum()*100/(n_columns*n_rows),3)))


Data quality completeness evaluation - ArticlesDataset

Number of rows: 2969
Number of columns: 4

Tuple completeness (in %)
100.0% tuple with 100% of completeness

Attribute completeness (in %)
username    100.0
day         100.0
title       100.0
inTitle     100.0
dtype: float64

Table completeness: 100.0%
